In [1]:
# change these to try this notebook out
BUCKET = 'huiyi-sandbox'
PROJECT = 'huiyi-training'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.7' 

Deploy trained model

Deploying the trained model to act as a REST web service is a simple gcloud call.

In [3]:
%bash
gsutil ls gs://${BUCKET}/tuto/trained_model/export/exporter/

gs://huiyi-sandbox/tuto/trained_model/export/exporter/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556264922/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556265520/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556266120/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556266720/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556267320/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556267922/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556268521/
gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556268945/


In [4]:
%bash
#gcloud ml-engine models create tuto_model
MODEL_NAME="tuto_model"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/tuto/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying tuto_model ml_on_gcp from gs://huiyi-sandbox/tuto/trained_model/export/exporter/1556268945/ ... this will take a few minutes


Created ml engine model [projects/huiyi-training/models/tuto_model].
Creating version (this might take a few minutes)......
.........................................................................done.


Use model to predict (online prediction)

Send a JSON request to the endpoint of the service to make it predict a sale price. The order of the responses are the order of the instances.

In [5]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'tuto_model'
MODEL_VERSION = 'ml_on_gcp'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)

headers = {'Authorization': 'Bearer ' + token }
data = {
  "instances": [
    {
    "key": "a1", 
    "BOROUGH": "2", 
    "BLOCK": 1133.0, 
    "ZIP_CODE": 10099.0
    },
    {
    "key": "a2", 
    "BOROUGH": "5", 
    "BLOCK": 2126.0, 
    "ZIP_CODE": 10175.0
    }
  ]
}

response = requests.post(api, json=data, headers=headers)
print(response.content)

{"predictions": [{"predictions": [1627683.375], "key": ["a1"]}, {"predictions": [1501837.625], "key": ["a2"]}]}


Use model to predict (batch prediction)

Batch prediction is commonly used when you thousands to millions of predictions. Create a file withe one instance per line and submit using gcloud.

In [6]:
%writefile inputs.json
{"key": "b1", "BOROUGH": "1", "BLOCK": 1003.0, "ZIP_CODE": 11199.0}
{"key": "b2", "BOROUGH": "4", "BLOCK": 2196.0, "ZIP_CODE": 10105.0}

Overwriting inputs.json


In [7]:
%bash
INPUT=gs://${BUCKET}/tuto/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/tuto/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ml-engine jobs submit prediction tutopred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=tuto_model --version=ml_on_gcp

jobId: tutopred_190426_114214
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  135.0 B/  135.0 B]                                                
Operation completed over 1 objects/135.0 B.                                      
Removing gs://huiyi-sandbox/tuto/batchpred/outputs/prediction.errors_stats-00000-of-00001#1556205802089369...
Removing gs://huiyi-sandbox/tuto/batchpred/outputs/prediction.results-00000-of-00001#1556205782831224...
/ [2/2 objects] 100% Done                                                       
Operation completed over 2 objects.                                              
Job [tutopred_190426_114214] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe tutopred_190426_114214

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs tutopred_190426_114214
